In [2]:
# 주피터 노트북 환경설정
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

from IPython.display import Image

from IPython.core.display import display, HTML
display(HTML("<style>.container { font-weight: bold !important; }</style>"))
display(HTML("<style>.container { width: 98% !important; }</style>"))

In [1]:
import numpy as np
import pandas as pd
import openpyxl

print('테이블 명세서 코드')

#엑셀 파일 불러오기
#데이터 처리
dbInfo = pd.read_excel('saleviscaretable.xlsx')
print(dbInfo.head())
## 반복문을 위한 테이블명을 조사 중복을 없앤다
dbInfo['테이블명'].duplicated()
## true == 중복, false == 최초의 값, 즉, false만 뽑으면 각 테이블명만 담아낼수 있다..

dbTableNames = dbInfo['테이블명'].drop_duplicates()


arrTableNames = np.array(dbTableNames.values.tolist())
print("------어레이에 담긴 테이블명들 확인-------")
print(arrTableNames)
print("----------------------------------------")
##테이블명의 인덱스 넘버
tableNameIndex = dbTableNames.index
#테이블명의 인덱스로 해당 테이블명의 논리테이블명을 가져온다
arrTableInfo =[]
for i in tableNameIndex:
    x = dbInfo.at[i, '논리테이블명']
    arrTableInfo.append(x)


# Workbook == 엑셀파일전체 AND Worksheet == 엑셀시트하나하나 
# spreadsheet 라는 변수에 .active 사용시 활성화된 시트를 불러온다. 초기엔 하나뿐..
# 시트이름 바꾸기 ex) sheet2.title = '수집 데이터'

# 원하는 셀에 값 입력하기.. 예) A1, B1, C1, D1 == '테이블 이름'

## 데이터프레임처럼 pd.read_excel 로도 엑셀파일을 불러올 수 있지만, 
## Workbook 의 메소드로도 기존 엑셀파일을 로드 할수 있다.
## 예) openpyxl.load_workbook(파일명) ==> wb=openpyxl.load_workbook('test.xlsx')
## 엑셀파일 저장은 Workbook.save(파일명) 으로 가능하다.
## 예) wb.save('test.xlsx')

## 엑셀 파일 만들기



def createWB(a):
    wb = openpyxl.Workbook()
    print(str(a) + "번째 엑셀시트 생성")
    sheet = wb.create_sheet('sheet' + str(a))
    sheet.title = '테이블명세서' + str(a)
    print(str(a) + "번째 엑셀시트의 헤드 작성")
    printTableHead(sheet)
    print(str(a) + "번째 엑셀시트의 값 작성")
    printTableValues(sheet,a)

    ##엑셀파일 저장
    wb.save(filename="test.xlsx")


## 테이블명세서의 헤더 만드는 함수.

def printTableHead(sheet):

    sheet.merge_cells('A1:D1') 
    sheet['A1']= '테이블 이름'
    sheet.merge_cells('E1:J1')
    sheet.merge_cells('A2:D2') 
    sheet['A2']= '테이블 설명'
    sheet.merge_cells('E2:J2')
    sheet.merge_cells('A3:D3') 
    sheet['A3']= 'PRIMARY KEY'
    sheet.merge_cells('E3:J3') 
    sheet.merge_cells('A4:D4') 
    sheet['A4']= 'FOREIGN KEY'
    sheet.merge_cells('A5:D5') 
    sheet['A5']= 'INDEX'
    sheet.merge_cells('A6:D6') 
    sheet['A6']= 'UNIQUE INDEX'
    sheet.merge_cells('E4:J4')
    sheet.merge_cells('E5:J5') 
    sheet.merge_cells('E6:J6') 
    sheet['A7']= 'NO'
    sheet['B7']= 'PK'
    sheet['C7']= 'AI'
    sheet['D7']= 'FK'
    sheet['E7']= 'NULL'
    sheet['F7']= '컬럼 이름'
    sheet['G7']= 'TYPE'
    sheet['H7']= 'DEFAULT'
    sheet['I7']= '설명'
    sheet['J7']= '참조 테이블'

##테이블의 값 넣는 함수
def printTableValues(sheet,a):
    print(str(a) + "값 insert 시작")

    i = arrTableNames[a]
    j = arrTableInfo[a]
    #테이블명 입력
    sheet['E1'] = i
    #논리테이블명입력
    sheet['E2'] = j
    ## 각 테이블명의 각각의 데이터프레임 새로 생성
    tableDetailByName = dbInfo.loc[dbInfo['테이블명'] == arrTableNames[a]]
    #PK조사
    thisPK = tableDetailByName.loc[tableDetailByName['KEY']=='PRI']
    PKString = thisPK['컬럼명'].values
    yStr=''
    if not PKString.all():
        print("PK 컬럼 없음")
    else:
        lenPK = len(PKString)
        for y in range(lenPK):
            yStr += thisPK['컬럼명'].values[y]

        #PK정보입력
        sheet['E3']= yStr
    #FK조사
    thisFK = tableDetailByName.loc[tableDetailByName['KEY']=='MUL']
    FKString = thisFK['컬럼명'].values
    #FK정보입력
    zStr=''
    if not FKString.all():
        print("FK컬럼 없음")
    else:
        lenFK = len(FKString)
        for z in range(lenFK):
            zStr += thisFK['컬럼명'].values[z]
        sheet['E4']= zStr

    #컬럼의 갯수조사 인덱스 설정
    nn = tableDetailByName.index +1
    #컬럼 인덱스 입력
    for i, value in enumerate(nn):
        sheet.cell(row=i+8, column=1, value=value)
    #컬럼명 조사
    columnNames = tableDetailByName['컬럼명']
    cn = columnNames.values
    #컬럼명 입력
    for i, value in enumerate(cn):
        sheet.cell(row=i+8, column=6, value=value)
    #데이터 형 조사
    dataTypes = tableDetailByName['데이터 길이']
    dt = dataTypes.values
    #데이터 형 입력
    for i, value in enumerate(dt):
        sheet.cell(row=i+8, column=7, value=value)
    #컬럼설명 조사
    columnDescs = tableDetailByName['컬럼설명']
    cd = columnDescs.values
    #컬럼설명 입력
    for i, value in enumerate(cd):
        sheet.cell(row=i+8, column=9, value=value)
    #눌 허용 조사
    isNull = tableDetailByName['NULL허용']
    isn = isNull.values
    # 눌 허용값 입력
    for i, value in enumerate(isn):
        sheet.cell(row=i+8, column=5, value=value)
    #디폴트값 조사
    isDefault = tableDetailByName['디폴트값']
    isd = isDefault.values
    isdS = isd.astype(str)

    #디폴트값 입력
    for i, value in enumerate(isdS):
        sheet.cell(row=i+8, column=8, value=value)
    
    ## PK와 FK 컬럼에 'Y' 표기
    ## .values 또는 to._numpy()  numpy배열로 변환함!
    ## 인덱싱으로 특정값을 변경한다. (Indexing: a[a < 0] = 0) 
    isThisPK = tableDetailByName['KEY'].to_numpy()
    isThisPK[isThisPK=='PRI']='Y'
    for i, value in enumerate(isThisPK):
        sheet.cell(row=i+8, column=2, value=value)
    
    isThisFK = tableDetailByName['KEY'].values
    isThisFK[isThisFK=='MUL'] = 'Y'
    isThisFK[isThisFK!='MUL'] = ''
    for i, value in enumerate(isThisFK):
        sheet.cell(row=i+8, column=4, value=value)
    ## 데이터프레임에 아예 변화를 주기때문에 엑셀화 할때 제일 마지막단계에서 실행해주고 다음
    ## 테이블로 넘어간다..


#테이블명의 length
lenarr = len(arrTableNames)


#for loop 을 이용하여 함수호출!

print("테이블명세서 작업 시작")
for a in range(lenarr):

    createWB(a)

테이블 명세서 코드
                   테이블명       논리테이블명  순번           컬럼명   데이터 타입        데이터 길이  \
0  additionalserviceext  게시판 사용자정의필드   1  RelateDataId   bigint    bigint(11)   
1  additionalserviceext  게시판 사용자정의필드   2  ExtVarchar01  varchar  varchar(100)   
2  additionalserviceext  게시판 사용자정의필드   3  ExtVarchar02  varchar  varchar(100)   
3  additionalserviceext  게시판 사용자정의필드   4  ExtVarchar03  varchar  varchar(100)   
4  additionalserviceext  게시판 사용자정의필드   5  ExtVarchar04  varchar  varchar(100)   

   KEY NULL허용 자동여부 디폴트값        컬럼설명  
0  PRI     NO  NaN  NaN       게시물ID  
1  NaN    YES  NaN  NaN  Varchar 01  
2  NaN    YES  NaN  NaN  Varchar 02  
3  NaN    YES  NaN  NaN  Varchar 03  
4  NaN    YES  NaN  NaN  Varchar 04  
------어레이에 담긴 테이블명들 확인-------
['additionalserviceext' 'aliastable' 'attachment' 'bbsmaster'
 'bbsmasterext' 'bbspost' 'bbspostcomm' 'bbspostext' 'bbspostread'
 'bookmark' 'campaign' 'campaigncustomer' 'campaigncustomerext'
 'campaignext' 'campaignimpl' 'campaignimplcustomer

PermissionError: [Errno 13] Permission denied: 'test.xlsx'